In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
# Validation
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score

In [35]:
train_df = pd.read_csv('Dataset/SmokerStatusPrediction/train.csv', encoding='utf-8')
test_df = pd.read_csv('Dataset/SmokerStatusPrediction/test.csv', encoding='utf-8')

In [36]:
# EDA session will be done in another notebook
# Training models
seed = np.random.seed(42)
X = train_df.drop(["id", "smoking"],axis=1)
y = train_df["smoking"]

In [40]:
test_cv = test_df.drop(columns = ['id'], axis = 1)

In [39]:
# Baseline xgboost model
xgbmodel = XGBClassifier(random_state=seed, tree_method='exact')
print("CV score of XGB is ",cross_val_score(xgbmodel, X, y, cv=4, scoring = 'roc_auc').mean())

CV score of LGBM is  0.8645777542659799


In [50]:
# Baseline xgboost model
xgbmodel_md = XGBClassifier(random_state=seed, tree_method='exact').fit(X, y)
print("CV score of XGB is ",cross_val_score(xgbmodel_md, X, y, cv=4, scoring = 'roc_auc').mean())

CV score of XGB is  0.8645777542659799


In [52]:
# Prediction
XGB_pred_test = xgbmodel_md.predict_proba(test_cv)[:, 1]
XGB_pred_test

array([0.65870994, 0.10153673, 0.6664567 , ..., 0.49006492, 0.0941251 ,
       0.03032043], dtype=float32)

In [53]:
# Generate submission file
submission_df = pd.read_csv(r"Dataset/SmokerStatusPrediction/sample_submission.csv", encoding="utf8")
submission_df['smoking'] = XGB_pred_test
submission_df

,id,smoking
0,159256,0.658710
1,159257,0.101537
2,159258,0.666457
3,159259,0.014916
4,159260,0.631548
...,...,...
106166,265422,0.514796
106167,265423,0.571328
106168,265424,0.490065
106169,265425,0.094125


In [54]:
submission_df.to_csv('Dataset/SmokerStatusPrediction/submissionBaselineXGB.csv', index=False)